In [1]:
import spacy
import numpy as np
import gzip
import json
import os
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import tokenize_uk
from langdetect import detect
import pymorphy2
from collections import Counter

In [2]:
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [ ]:
nlp_uk = spacy.load('/tmp/uk_vectors/news_cased_tokenized_word2vec')

In [ ]:
nlp_uk = spacy.load('/tmp/uk_vectors/fiction_lowercased_tokenized_word2vec')

In [ ]:
nlp_uk = spacy.load('/tmp/uk_vectors/news_lowercased_lemmatized_word2vec')

In [ ]:
nlp_uk = spacy.load('/tmp/uk_vectors/ubercorpus_lowercased_tokenized_word2vec')

In [441]:
# for some reason makes it worse, TODO: play with it and finf a way
nlp_uk = spacy.load('/tmp/uk_vectors/ubercorpus_lowercased_lemmatized_word2vec') # used for grouping cats

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py:193: UserWarning: [W019] Changing vectors name from uk_model.vectors to uk_model.vectors_325250, to avoid clash with previously loaded vectors. See Issue #3853.
  "__main__", mod_spec)


In [ ]:
# nlp_uk = spacy.load('/tmp/uk_vectors/ubercorpus_cased_tokenized_word2vec') # no

In [ ]:
# nlp_uk = spacy.load('/tmp/uk_vectors/news_lowercased_tokenized_word2vec') # no

In [ ]:
# nlp_uk = spacy.load('/tmp/uk_vectors/ubercorpus_cased_lemmatized')

In [435]:
nlp_uk = spacy.load('/tmp/uk_vectors/ubercorpus_lowercased_tokenized') # for classification

In [4]:
def open_file(filename):
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        j = json.load(f)
        return j[0]['CallZText'], j[0]['CallZType']

In [490]:
PATH = '../../../../1551.gov.ua/raw/'
def read_files():
    data = []
    for d in os.listdir(PATH):
        files = os.listdir(PATH + '/' + d)
        for file in files:
            data.append(open_file(os.path.join(PATH + '/' + d + f'/{file}')))
    return data


def get_categories(data):
    return [y for _, y in data]


def get_claims(data, cats):
    return [x for x, y in data if y in cats]


def lowerize(text):
    words = tokenize_uk.tokenize_uk.tokenize_words(text)
    return ' '.join([x.lower() for x in words])

# STOP POINT!!!
# TODO:
# - figure out why lower/lemma don't work
# - find better vectors (or compare with existing)
def vectorize_sent(v_unk, text, preprocessor):
#     text_lower = ' '.join([x.lower() for x in text.split()])
#     text_lemma = ' '.join([morph.parse(x)[0].normal_form for x in text.split()])
# tokenize_uk.tokenize_uk.tokenize_words(text)
    
    v = 0
    text_processed = preprocessor(text)
    
#     doc = nlp_uk(text_processed)
    doc = nlp_uk(text_processed)
    total_len = len(doc)
    for tok in doc:
        # like_num
#         if not tok.is_stop:
        v += tok.vector
#         v /= len(doc)
    return v


def get_claim_label(claim_pair, cat_groups):
    label = 'Інше'
    claim, current_label = claim_pair
    for k, v in cat_groups.items():
        if current_label in v:
            label = k
    return label


def get_data(data, cat_groups):
    stop = ['вже', 'коли', 'ані', 'над', 'по', 'перед', 'після', 'або', 'вже']
    claims = []
    labels = []
    for x, y in data:
        try:
            lang = detect(x)
            if lang == 'uk': # or lang == 'ru':
#                 words = tokenize_uk.tokenize_uk.tokenize_words(x)
#                 words = [x for x in words]
#                 words = [morph.parse(x)[0].normal_form for x in words if x not in stop]
#                 t = ' '.join(words)
#                 claims.append(t)
                label = get_claim_label((x, y), cat_groups)
                claims.append(x)
#                 claims.append(t)
                labels.append(label)
        except:
            pass
            
    return claims, labels


def get_all_cat_nlps(cats):
    return [nlp_uk(x) for x in cats]

def normalize_cat(words):
        norm = []
        for w in words:
            m = morph.parse(w)[0]
            if m.tag.POS and m.tag.POS != 'PREP' and m.tag.POS != 'PRCL' and m.tag.POS != 'NUMR':
                norm.append(m.normal_form)

        return nlp_uk(' '.join(norm))
    

def is_similar_cats(cat1, cat2):
    
    
    def _is_water(w1, w2):
        waters = ['ХВП', 'ГВП']
        return any(x.upper() in waters for x in w1) and any(x.upper() in waters for x in w2)
    
    def _is_dism(w1, w2):
        dism = 'відсутність опалення'
        return set([x.lower() for x in w1]).intersection(dism) and set([x.lower() for x in w2]).intersection(dism)
    
    def _is_lift(w1, w2):
        lift = 'ліфт'
        return lift in [x.lower() for x in w1] and lift in [x.lower() for x in w2]
    
    def _is_inst(w1, w2):
        keyword = 'встановлення'
        return keyword in [x.lower() for x in w1] and keyword in [x.lower() for x in w2]
    
    if not cat1 or not cat2:
        return False
    if not cat1.vector_norm or not cat2.vector_norm:
        min_len = min(len(c1), len(c2))
        if min_len < 3:
            return False
        
        skip_pos = ['CONJ']
        morph1 = [morph.parse(x.text)[0] for x in c1]
        morph2 = [morph.parse(x.text)[0] for x in c2]
        inter = set([x.normal_form for x in morph1 if x.tag.POS not in skip_pos]).\
                intersection([x.normal_form for x in morph2 if x.tag.POS not in skip_pos])
        return len(inter)/min_len > 0.6
        
    w1 = tokenize_uk.tokenize_uk.tokenize_words(cat1)
    w2 = tokenize_uk.tokenize_uk.tokenize_words(cat2)
    
    if _is_water(w1, w2) or _is_dism(w1, w2) or _is_lift(w1, w2) or _is_inst(w1, w2):
        return True
    sim = normalize_cat(w1).similarity(normalize_cat(w2))
#     sim = cat1.similarity(cat2)
    return sim > 0.6


def get_common_group_name(docs):
    others = []
    fst_sent = docs[0].text
    words = fst_sent.split()
    if 'ХВП' in words or 'ГВП' in words:
        fst_sent = 'Проблеми з ХВП/ГВП'
    return fst_sent


def group_categories(cats, compare_fn):
    grouped = []
    cache = {}
    
    def _compare(a, b):
        if not cache.get((a, b)):
            is_sim = compare_fn(a, b)
            cache[(a, b)] = is_sim
            cache[(b, a)] = is_sim
        else:t
            is_sim = cache[(a, b)]
        return is_sim

    def _is_most_comp(lst, item):
        similars = [x for x in lst if _compare(item, x)]
        if len(lst) < 3:
            return len(similars) == len(lst)
        if len(lst) == 3:
            return len(similars)/len(lst) > 0.6
        return len(similars)/len(lst) >= 0.4
    

    def _inner(cts):
        rst = []
        processed = []

        if len(cts) == 1:
            a, b = cts[0], None
        elif len(cts) > 2:
            a, b, *rest = cts
            rst = rest
        elif len(cts) == 2:
            a, b = cts
        
        if len(grouped):
            for gr in grouped:
                is_a = _is_most_comp(gr, a)
                is_b = _is_most_comp(gr, b)
                if is_a or is_b:
                    idx = grouped.index(gr)
                    
                    if is_a:
                        grouped[idx].append(a)
                        processed.append(a)
                    if is_b:
                        grouped[idx].append(b)
                        processed.append(b)
        if a not in processed or b not in processed:
            if _compare(a, b):
                if a not in processed and b in processed:
                    grouped.append([a])
                elif a in processed and b not in processed:
                    grouped.append([b])
                else:
                    grouped.append([a, b])
            else:
                if a and a not in processed:
                    grouped.append([a])
                if b and b not in processed:
                    grouped.append([b])
        if rst:
            ll = len(rst)
            if ll % 10 == 0:
                print(ll)
            _inner(rst)
    _inner(cats)
    
    return grouped


def get_calegory_groups(cats, compare_fn):
    res = {}
    cats_grouped = group_categories(cats, compare_fn)
    for gr in cats_grouped:
        group_name = get_common_group_name(gr)
        res[group_name] = gr
    return res

def get_groups_text(cats_grouped):
    res = {}
    for k, v in cats_grouped.items():
        res[k] = [x.text for x in v]
    return res

def reduce_cats(cats):
    res = {'Інше': 0}
#     res = {}
    for x, y in cats:
        if y > 200:
            res[x] = y
        else:
            res['Інше'] += y
    return res

def get_all_claims(data):
    return [x for x, _ in data]

def get_all_claims_labels(data, categories_grouped):
    return [get_claim_label(x, categories_grouped) for x in data]

In [35]:
def print_report(data, cats, v_unk, preprocessor):
    all_claims, all_labels = get_data(data, cats)
    X = [vectorize_sent(v_unk, x, preprocessor) for x in all_claims]
    y = all_labels
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    clf.fit(X_train, y_train)
    print(classification_report(y_test, clf.predict(X_test)))

In [6]:
data = read_files()

In [179]:
all_claim_categories = sorted(Counter(get_categories(data)).items(), key=lambda x: x[1], reverse=True)
all_categories = sorted(set(reduce_cats(all_claim_categories)))
all_cat_nlps = get_all_cat_nlps(all_categories)

In [491]:
categories_grouped_nlp = get_calegory_groups(all_cat_nlps, is_similar_cats)
categories_grouped = get_groups_text(categories_grouped_nlp)

>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>> cache
>

In [15]:
v_unk = nlp_uk('unk')[0].vector

In [296]:
clf = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
                     weights='uniform')

In [440]:
print_report(data[:1000], categories_grouped, v_unk, lowerize)

                                                                                precision    recall  f1-score   support

                                                                          Інше       0.31      0.57      0.40        40
                                                                   Інші Подяки       0.00      0.00      0.00         1
                                            Інші технічні недоліки стану ліфту       0.14      0.25      0.18         4
              Аварійний, травмонебезпечний стан утримання об’єктів благоустрою       0.14      0.17      0.15         6
                                Будівництво/дооблаштування дитячого майданчику       0.00      0.00      0.00         2
                              Видалення аварійних, пошкоджених хворобами дерев       0.00      0.00      0.00         0
                                                    Вилов безпритульних тварин       0.67      1.00      0.80         2
                                       

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [493]:
# TODO: there are duplicates under diff cats!
# categories_grouped

with open('./cats_gr.json', 'w') as f:
    json.dump(categories_grouped, f, ensure_ascii=False)

True

In [492]:
len(categories_grouped)

53

In [469]:
sents = [
        "Відсутність каналізаційних люків та решіток на проїжджих частинах",
        "Відсутність каналізаційних, водопровідних люків",
        "Обслуговування та технічний стан каналізаційних колодязів",
        "Технічний стан проїжджих частин вулиць та тротуарів",
        "Технічний стан підземних пішохідних переходів"
    ]

c1 = nlp_uk(sents[0])
c2 = nlp_uk(sents[1])
c3 = nlp_uk(sents[2])
c4 = nlp_uk(sents[3])
c5 = nlp_uk(sents[4])
c1.similarity(c5)

cache = {}
cache[(c1, c2)] = True
cache[(c1, c2)]

True

In [425]:
def aa(c1, c2):
    skip_pos = ['CONJ']
    morph1 = [morph.parse(x.text)[0] for x in c1]
    morph2 = [morph.parse(x.text)[0] for x in c2]
    inter = set([x.normal_form for x in morph1 if x.tag.POS not in skip_pos]).\
            intersection([x.normal_form for x in morph2 if x.tag.POS not in skip_pos])
    print(inter, len(inter)/min(len(c1), len(c2)))
    return len(inter)/min(len(c1), len(c2)) >= 0.6

In [426]:
aa(c3, c4)

{'технічний', 'стан'} 0.3333333333333333


False

In [422]:
for tok in c4:
    print(tok, morph.parse(tok.text)[0].tag.POS)

Технічний ADJF
стан NOUN
проїжджих ADJF
частин NOUN
вулиць NOUN
та CONJ
тротуарів NOUN


In [ ]:
  def _compare(a, b):
        if not cache.get((a, b)):
            is_sim = compare_fn(a, b)
            cache[(a, b)] = is_sim
            cache[(b, a)] = is_sim
        else:
            print('>> cache')
            is_sim = cache[(a, b)]